In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime, timedelta
from pathlib import Path

import copy
import pyarrow.parquet as pq

In [2]:
from technicals import *
from tsfresh import extract_features

In [3]:
data_path = Path('../data')
data_path.exists()

True

<h3> concatenate book data for train and test</h3>

In [4]:
dest_folder ='z_2'

In [5]:
trade_train_path = data_path.joinpath('trade_train.parquet')
trade_train_sub = next(os.walk(trade_train_path))[1]

In [6]:
for sub_f_ in trade_train_sub:
    file_loc= trade_train_path.joinpath(sub_f_)
    df = pq.read_table(file_loc).to_pandas()
    
    df['avg_size']=(df['size']/df['order_count']).round(1)

    #cal technical indicators by time_id
    sum_list = []
    for time_id_ in df['time_id'].unique().tolist():
        series_ = df.loc[df['time_id']==time_id_, 'price']
        item_ = [time_id_, cal_ema(series_), cal_rsi(series_), cal_ssma(series_), cal_dema(series_), 
                 cal_tema(series_), cal_trix(series_), cal_er(series_), cal_smma(series_), 
                 cal_roc(series_), cal_stochrsi(series_)]
        sum_list.append(item_)

    header =['time_id', 'price_ema', 'price_rsi', 'price_ssma', 'price_dema', 
                 'price_tema', 'price_trix', 'price_er', 'price_smma', 
                 'price_roc', 'price_stochrsi']
    df_pivot = pd.DataFrame(sum_list, columns=header)

    #cal realized volatility
    df['log_return']=df.groupby(['time_id'])['price'].apply(cal_log_return)
    df_ = df.loc[df['log_return'].notna(), ['time_id', 'log_return']].copy(deep=True)
    df_real_vol_ =  df_.groupby(['time_id'])['log_return'].agg(cal_real_vol).reset_index()
    df_real_vol_.rename(columns = {'log_return':'real_vol'}, inplace=True)

    print(df_pivot.shape)
    df_pivot = df_pivot.merge(df_real_vol_, how='inner', on=['time_id'])
    print(df_pivot.shape)
    del df_real_vol_, df_

    #calculate basic summary stats
    #min, max, med
    df_ = df.pivot_table(index=['time_id'], values=['price', 'size', 'order_count', 'avg_size'], 
                   aggfunc={np.mean, np.median, np.max, np.min, np.std
                           }
                  )

    new_cols = []
    for c_ in df_.columns:
        new_cols.append(f'{c_[0]}_{c_[1]}')

    df_.columns=new_cols
    df_.reset_index(inplace=True)


    print(df_pivot.shape)
    df_pivot = df_pivot.merge(df_, how='inner', on=['time_id'])
    print(df_pivot.shape)
    del df_

    #get tsfresh features
    sel_cols=['time_id', 'seconds_in_bucket', 'price']
    df_features = extract_features(df[sel_cols], column_id="time_id", column_sort="seconds_in_bucket")
    df_features['time_id']=df_features.index

    #merge all into one dataframe
    print(df_features.shape, df_pivot.shape)
    df_pivot = df_pivot.merge(df_features, on='time_id', how='inner')
    print(df_pivot.shape)
    del df_features


    #assign stock id and file
    stock_id=sub_f_.split('=')[-1]  
    df_pivot['stock_id']=stock_id        
    df_pivot['file']=str(file_loc)

    #save data
    df_pivot.to_csv(f'../data/{dest_folder}/{stock_id}', sep='|', compression='bz2')

    del df_pivot, df

(3830, 11)
(3830, 12)
(3830, 12)
(3830, 32)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:39<00:00,  1.96s/it]


(3830, 788) (3830, 32)
(3830, 819)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 32)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:46<00:00,  2.35s/it]


(3830, 788) (3830, 32)
(3830, 819)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 32)


...

(3830, 788) (3830, 32)
(3830, 819)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 32)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:40<00:00,  2.00s/it]


(3830, 788) (3830, 32)
(3830, 819)
(3830, 11)
(3830, 12)
(3830, 12)
(3830, 32)


Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 20/20 [00:54<00:00,  2.71s/it]


(3830, 788) (3830, 32)
(3830, 819)
